In [34]:
import pandas as pd
import pickle
import copy
warf = pd.read_csv('data/warfarin.csv')

In [35]:
dummies = ['Gender', 
           'Race', 
           'Ethnicity', 
           'Age',
           'Cyp2C9 genotypes',
           'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T',
           'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C',    
           'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G',
           'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G']
remove_the_na = ['Rifampin or Rifampicin', 'Macrolide Antibiotics']
remove_the_val = ['Cerivastatin (Baycol)']
dummies_val_and_na = ['Diabetes', 
                      'Congestive Heart Failure and/or Cardiomyopathy',
                      'Valve Replacement', 
                      'Aspirin',
                      'Acetaminophen or Paracetamol (Tylenol)',
                      'Was Dose of Acetaminophen or Paracetamol (Tylenol) >1300mg/day',
                      'Simvastatin (Zocor)',
                      'Atorvastatin (Lipitor)',
                      'Fluvastatin (Lescol)',
                      'Lovastatin (Mevacor)',
                      'Pravastatin (Pravachol)',
                      'Rosuvastatin (Crestor)',
                      'Amiodarone (Cordarone)',
                      'Carbamazepine (Tegretol)',
                      'Phenytoin (Dilantin)',
                      'Sulfonamide Antibiotics',
                      'Anti-fungal Azoles',
                      'Herbal Medications, Vitamins, Supplements',
                      'Current Smoker']
height_and_weight = ['Height (cm)', 
                      'Weight (kg)']

cols = copy.deepcopy(dummies)
# cols.extend(dummies_with_na)
cols.extend(remove_the_na)
cols.extend(remove_the_val)
cols.extend(dummies_val_and_na)
cols.extend(height_and_weight)
cols.append("Therapeutic Dose of Warfarin")
cols.append('Indication for Warfarin Treatment')
warf = warf[cols]

In [36]:
# ones with no NA
warf = pd.get_dummies(warf, prefix=dummies, columns=dummies, dummy_na=False)
# ones with NA
# warf = pd.get_dummies(warf, prefix=dummies_with_na, columns=dummies_with_na,dummy_na=True )
cols = list(warf)
cols.remove('Race_Unknown')
cols.remove('Ethnicity_Unknown')
warf = warf[cols]

In [37]:
for col in remove_the_na:
    warf[col] = warf.fillna(0)[col]
for col in height_and_weight:
    warf[col] = warf.fillna(0)[col]
warf = warf.fillna('Unknown')
warf = warf.loc[warf["Therapeutic Dose of Warfarin"] != 'Unknown']
warf = warf.reset_index(drop=True)

true_value_numbers = warf["Therapeutic Dose of Warfarin"]
with open('data/true_dosage.pkl', 'wb') as f:
    pickle.dump(true_value_numbers, f)
len(warf)

5528

In [38]:
for col in remove_the_val: 
    warf[col] = warf[col] == 'Unknown'

In [39]:
warf = pd.get_dummies(warf, prefix=dummies_val_and_na, columns=dummies_val_and_na)
cols = list(warf)
for col in dummies_val_and_na:  
    cols.remove(col + '_0.0')

warf = warf[cols]

In [40]:
for col in height_and_weight:
    warf[col + '_NA'] = warf[col] == 0
# warf[col] = (warf[col] - warf[col].mean())/warf[col].std()

In [41]:
list(warf)
for i in range(1, 9, 1):
    warf['Indication for Warfarin Treatment' + '_' + str(i)] = \
    warf['Indication for Warfarin Treatment'].str.contains(str(i))
warf['Indication for Warfarin Treatment' + '_NA'] = warf['Indication for Warfarin Treatment'] == "Unknown"

In [42]:
cols = list(warf)
cols.remove('Indication for Warfarin Treatment')
warf = warf[cols]

Save the true dosage values

In [43]:
low = warf.loc[warf["Therapeutic Dose of Warfarin"] < 21]
medium = warf.loc[warf["Therapeutic Dose of Warfarin"] >= 21]
medium = medium.loc[medium["Therapeutic Dose of Warfarin"] <= 49]
high = warf.loc[warf["Therapeutic Dose of Warfarin"] > 49]

max_low = low["Therapeutic Dose of Warfarin"].std()
min_low = low["Therapeutic Dose of Warfarin"].min()
max_med = medium["Therapeutic Dose of Warfarin"].std()
min_med = medium["Therapeutic Dose of Warfarin"].min()
max_high = high["Therapeutic Dose of Warfarin"].std()
min_high = high["Therapeutic Dose of Warfarin"].min()

warf["Therapeutic Dose of Warfarin"] = pd.cut(warf["Therapeutic Dose of Warfarin"]\
                                              , [0.0,20.9999,49.0,20000.0]).astype(str)
warf["Therapeutic Dose of Warfarin"] = warf["Therapeutic Dose of Warfarin"].map({'(0.0, 21.0]': "low", '(21.0, 49.0]': "medium", '(49.0, 20000.0]': "high"})

warf["Therapeutic Dose of Warfarin"].value_counts()
true_values = warf['Therapeutic Dose of Warfarin']
warf["Therapeutic Dose of Warfarin"] = warf["Therapeutic Dose of Warfarin"].map({"low": 0, "medium":1, "high":2})
# warf = warf.drop(columns='Therapeutic Dose of Warfarin')

Change the data to suit categories etc.

In [44]:
print(max_low)
print(min_low)
print(max_med)
print(min_med)
print(max_high)
print(min_high)

4.031703103381195
2.1
7.785291417550073
21.0
18.35817976658349
49.42


In [11]:
cols = [
    'Rifampin or Rifampicin',
 'Macrolide Antibiotics',
 'Cerivastatin (Baycol)',
 'Height (cm)',
 'Weight (kg)',
'Height (cm)_NA',
    'Weight (kg)_NA',
 'Gender_female',
 'Gender_male',
 'Race_Asian',
 'Race_Black or African American',
 'Race_White',
 'Ethnicity_Hispanic or Latino',
 'Ethnicity_not Hispanic or Latino',
 'Age_10 - 19',
 'Age_20 - 29',
 'Age_30 - 39',
 'Age_40 - 49',
 'Age_50 - 59',
 'Age_60 - 69',
 'Age_70 - 79',
 'Age_80 - 89',
 'Age_90+',
 'Cyp2C9 genotypes_*1/*1',
 'Cyp2C9 genotypes_*1/*11',
 'Cyp2C9 genotypes_*1/*13',
 'Cyp2C9 genotypes_*1/*14',
 'Cyp2C9 genotypes_*1/*2',
 'Cyp2C9 genotypes_*1/*3',
 'Cyp2C9 genotypes_*1/*5',
 'Cyp2C9 genotypes_*1/*6',
 'Cyp2C9 genotypes_*2/*2',
 'Cyp2C9 genotypes_*2/*3',
 'Cyp2C9 genotypes_*3/*3',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_A/A',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_A/G',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_G/G',
 'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C_G/G',
 'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C_G/T',
 'VKORC1 genotype: 497T>G (5808); chr16:31013055; rs2884737; A/C_T/T',
 'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G_C/C',
 'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G_C/G',
 'VKORC1 genotype: 1542G>C (6853); chr16:31012010; rs8050894; C/G_G/G',
 'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G_A/A',
 'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G_A/G',
 'VKORC1 genotype: 3730 G>A (9041); chr16:31009822; rs7294;  A/G_G/G',
 'Diabetes_1.0',
 'Diabetes_Unknown',
 'Congestive Heart Failure and/or Cardiomyopathy_1.0',
 'Congestive Heart Failure and/or Cardiomyopathy_Unknown',
 'Valve Replacement_1.0',
 'Valve Replacement_Unknown',
 'Aspirin_1.0',
 'Aspirin_Unknown',
 'Acetaminophen or Paracetamol (Tylenol)_1.0',
 'Acetaminophen or Paracetamol (Tylenol)_Unknown',
 'Was Dose of Acetaminophen or Paracetamol (Tylenol) >1300mg/day_1.0',
 'Was Dose of Acetaminophen or Paracetamol (Tylenol) >1300mg/day_Unknown',
 'Simvastatin (Zocor)_1.0',
 'Simvastatin (Zocor)_Unknown',
 'Atorvastatin (Lipitor)_1.0',
 'Atorvastatin (Lipitor)_Unknown',
 'Fluvastatin (Lescol)_1.0',
 'Fluvastatin (Lescol)_Unknown',
 'Lovastatin (Mevacor)_1.0',
 'Lovastatin (Mevacor)_Unknown',
 'Pravastatin (Pravachol)_1.0',
 'Pravastatin (Pravachol)_Unknown',
 'Rosuvastatin (Crestor)_1.0',
 'Rosuvastatin (Crestor)_Unknown',
 'Amiodarone (Cordarone)_1.0',
 'Amiodarone (Cordarone)_Unknown',
 'Carbamazepine (Tegretol)_1.0',
 'Carbamazepine (Tegretol)_Unknown',
 'Phenytoin (Dilantin)_1.0',
 'Phenytoin (Dilantin)_Unknown',
 'Sulfonamide Antibiotics_1.0',
 'Sulfonamide Antibiotics_Unknown',
 'Anti-fungal Azoles_1.0',
 'Anti-fungal Azoles_Unknown',
 'Herbal Medications, Vitamins, Supplements_1.0',
 'Herbal Medications, Vitamins, Supplements_Unknown',
 'Current Smoker_1.0',
 'Current Smoker_Unknown',
 'Indication for Warfarin Treatment_1',
 'Indication for Warfarin Treatment_2',
 'Indication for Warfarin Treatment_3',
 'Indication for Warfarin Treatment_4',
 'Indication for Warfarin Treatment_5',
 'Indication for Warfarin Treatment_6',
 'Indication for Warfarin Treatment_7',
 'Indication for Warfarin Treatment_8',
 'Indication for Warfarin Treatment_NA'
]
warf['Bias'] = 1 
cols.append('Bias')
warf = warf[cols]

In [16]:
warf.loc[warf["Weight (kg)"] > 0].mean()

Rifampin or Rifampicin                                 0.000761
Macrolide Antibiotics                                  0.001142
Cerivastatin (Baycol)                                  0.611872
Height (cm)                                          141.257930
Weight (kg)                                           77.846423
Height (cm)_NA                                         0.159627
Weight (kg)_NA                                         0.000000
Gender_female                                          0.415335
Gender_male                                            0.583904
Race_Asian                                             0.291096
Race_Black or African American                         0.086568
Race_White                                             0.534247
Ethnicity_Hispanic or Latino                           0.008371
Ethnicity_not Hispanic or Latino                       0.800419
Age_10 - 19                                            0.002664
Age_20 - 29                             

In [12]:
warf.to_pickle('data/cleaned_warfarin.pkl')
with open('data/true_labels.pkl', 'wb') as f:
    pickle.dump(true_values, f)

In [13]:
len(list(warf))

94

In [14]:
list(warf)

['Rifampin or Rifampicin',
 'Macrolide Antibiotics',
 'Cerivastatin (Baycol)',
 'Height (cm)',
 'Weight (kg)',
 'Height (cm)_NA',
 'Weight (kg)_NA',
 'Gender_female',
 'Gender_male',
 'Race_Asian',
 'Race_Black or African American',
 'Race_White',
 'Ethnicity_Hispanic or Latino',
 'Ethnicity_not Hispanic or Latino',
 'Age_10 - 19',
 'Age_20 - 29',
 'Age_30 - 39',
 'Age_40 - 49',
 'Age_50 - 59',
 'Age_60 - 69',
 'Age_70 - 79',
 'Age_80 - 89',
 'Age_90+',
 'Cyp2C9 genotypes_*1/*1',
 'Cyp2C9 genotypes_*1/*11',
 'Cyp2C9 genotypes_*1/*13',
 'Cyp2C9 genotypes_*1/*14',
 'Cyp2C9 genotypes_*1/*2',
 'Cyp2C9 genotypes_*1/*3',
 'Cyp2C9 genotypes_*1/*5',
 'Cyp2C9 genotypes_*1/*6',
 'Cyp2C9 genotypes_*2/*2',
 'Cyp2C9 genotypes_*2/*3',
 'Cyp2C9 genotypes_*3/*3',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_A/A',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_A/G',
 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_G/G',
 'VKORC1 genotype: 4